# Prepare data for Human Microbiota Bank Website

import required packages

In [46]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from Bio import SeqIO
from io import StringIO
from collections import Counter
import pandas as pd
import os
import sys
import shutil
import re
import pickle
import numpy as np

## Generate species list table

Species will include the in the literature (From Dr. Xianzhi Jiang) and those identified by Dr. Zidan Zhao

### humanRef

In [47]:
file_human_ref = '/mnt/d/linux/W/NCBI/MoonRef20180810/HumanRef2018.speciesLineageName'
df_taxid_humanRef_lineage = pd.read_csv(file_human_ref,sep='\t',dtype=str)
df_taxid_humanRef_lineage.head()

,tax_id,species_id,name,species,genus,family,order,class,phylum,kingdom
0,100053,100053,Leptospira alexanderi,Leptospira alexanderi,Leptospira,Leptospiraceae,NaN,Spirochaetia,Spirochaetes,Bacteria
1,1000560,562,Escherichia coli VDP,Escherichia coli,Escherichia,Enterobacteriaceae,Enterobacterales,Gammaproteobacteria,Proteobacteria,Bacteria
2,1000561,287,Pseudomonas aeruginosa AES-1R,Pseudomonas aeruginosa,Pseudomonas,Pseudomonadaceae,Pseudomonadales,Gammaproteobacteria,Proteobacteria,Bacteria
3,1000567,216816,Bifidobacterium longum 3_1_37DFAAB,Bifidobacterium longum,Bifidobacterium,Bifidobacteriaceae,Bifidobacteriales,Actinobacteria,Actinobacteria,Bacteria
4,1000570,1328,Streptococcus anginosus SK52 = DSM 20563,Streptococcus anginosus,Streptococcus,Streptococcaceae,Lactobacillales,Bacilli,Firmicutes,Bacteria


In [48]:
df_taxid_humanRef_lineage.shape

(31770, 10)

### Identified by MoonBiotech

In [49]:
file_moonbiotech = '/mnt/d/linux/M/www/Django/moonbio/media/HumanMicrobes/20180820MNH_human_micro_all.xlsx'
df_moonbiotech = pd.read_excel(file_moonbiotech,sheet_name='all',index_col=0,dtype=str)
df_moonbiotech.shape

(13608, 57)

In [50]:
df_moonbiotech.columns

Index(['all_IDs', 'seq', 'seq_len', 'Analysis', 'Notes', 'HM_subject',
       'HM_matchLength', 'HM_identical', 'HM_qcover', 'HM_identity',
       'HM_taxID', 'HM_name', 'HM_species_id', 'HM_species', 'HM_genus',
       'HM_family', 'HM_order', 'HM_class', 'HM_phylum', 'HM_kingdom',
       'nt_subject', 'nt_matchLength', 'nt_identical', 'nt_qcover',
       'nt_identity', 'nt_taxID', 'nt_name', 'nt_species_id', 'nt_species',
       'nt_genus', 'nt_family', 'nt_order', 'nt_class', 'nt_phylum',
       'nt_kingdom', 'MatchMNH', 'MatchMNH_len', 'Isolation sources code',
       'Isolation place', 'Isolation source', 'Isolation medium',
       'Cultivated medium', 'Amplified cultre medium', 'Cultivated TM',
       'cultivated condition', 'pH', 'Sequencing', 'Preservation Freezer 1',
       'Preservation Freezer 1 Box', 'Preservation Freezer 2',
       'Preservation Freezer 2 Box', 'Preserved copy 1', 'Preserved copy 2',
       'Preserved copy 3', 'Preserved copy 4', 'remark', 'auditor'],
    

In [51]:
df_moonbiotech.head()

,all_IDs,seq,seq_len,Analysis,Notes,HM_subject,HM_matchLength,HM_identical,HM_qcover,HM_identity,...,Preservation Freezer 1,Preservation Freezer 1 Box,Preservation Freezer 2,Preservation Freezer 2 Box,Preserved copy 1,Preserved copy 2,Preserved copy 3,Preserved copy 4,remark,auditor
index,,,,,,,,,,,,,,,,,,,,,
1,MNH00001,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,7,①,大-20,①,nan,nan,Y,Y,nan,nan
2,MNH00002,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,7,①,大-20,①,nan,nan,Y,nan,nan,nan
3,MNH00003,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,7,①,大-20,①,nan,nan,Y,nan,nan,nan
4,MNH00004,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,7,①,大-20,①,nan,nan,Y,Y,nan,nan
5,MNH00005,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,7,①,大-20,①,nan,nan,Y,Y,nan,nan


In [52]:
df_moonbiotech1 = df_moonbiotech[df_moonbiotech['seq']!='nan']
df_moonbiotech1.shape

(8282, 57)

In [53]:
df_moonbiotech1.head()

,all_IDs,seq,seq_len,Analysis,Notes,HM_subject,HM_matchLength,HM_identical,HM_qcover,HM_identity,...,Preservation Freezer 1,Preservation Freezer 1 Box,Preservation Freezer 2,Preservation Freezer 2 Box,Preserved copy 1,Preserved copy 2,Preserved copy 3,Preserved copy 4,remark,auditor
index,,,,,,,,,,,,,,,,,,,,,
141,MNH00141,GCTTACACATGCAGTCGAGGGGCAGCATGACTTAGCAATAGGTTGA...,1403,nan,nan,ABJL02000008.3336138.3337648,1405,1399,100,99.573,...,7,1,nan,nan,nan,nan,nan,nan,nan,nan
142,MNH00142,CTTACACATGCAGTCGAGGGGCAGCACGATGTAGCAATACATTGGT...,1407,nan,nan,AQHV01000028.191627.193141,1409,1406,100,99.787,...,7,1,nan,nan,nan,nan,nan,nan,nan,nan
145,MNH00145,GCTTACACATGCAGTCGAGGGGCAGCATTTTAGTTTGCTTGCAAAC...,1410,nan,nan,EQ973383,1413,1390,100,98.37200000000001,...,7,1,nan,nan,nan,nan,nan,nan,nan,nan
147,MNH00147,TACACATGCAGTCGAGGGGCAGCATTTTAGTTTGCTTGCAAACTAA...,1343,nan,nan,KP944117.1,1338,1335,99,99.77600000000001,...,7,1,nan,nan,nan,nan,nan,nan,nan,nan
148,MNH00148,CTTACACATGCAGTCGAGGGGCAGCATGATTTGTAGCAATACAGAT...,1409,nan,nan,AAXE02000112,1411,1409,100,99.85799999999999,...,7,1,nan,nan,nan,nan,nan,nan,nan,nan


In [54]:
df_moonbiotech1=df_moonbiotech1.copy()
df_moonbiotech1['seq_len'] = df_moonbiotech1['seq_len'].astype(int)

In [55]:
df_moonbiotech2 = df_moonbiotech1[df_moonbiotech1['seq_len']>1000]
df_moonbiotech2.shape

(7323, 57)

find representive ids

In [56]:
st_moon_rep = set(df_moonbiotech2['MatchMNH'])
print(len(st_moon_rep))

4765


In [57]:
# get represntive lineages
df_moonbiotech3 = df_moonbiotech2[df_moonbiotech2['all_IDs'].apply(lambda x:x in st_moon_rep)]
df_moonbiotech3.shape

(4765, 57)

In [58]:
# get moon species not in df_taxid_humanRef_lineage
st_humanRef_tax = set(df_taxid_humanRef_lineage['species_id'])
ls_moon_unique_speciesIDs = [e for e in df_moonbiotech3['HM_species_id'] if e not in st_humanRef_tax]
print(len(set(ls_moon_unique_speciesIDs)),len(ls_moon_unique_speciesIDs))

28 115


In [59]:
df_taxid_moon = df_moonbiotech3.loc[:,['HM_taxID', 'HM_name', 'HM_species_id', 'HM_species', 'HM_genus',
                                       'HM_family', 'HM_order', 'HM_class', 'HM_phylum', 'HM_kingdom']]

In [60]:
df_taxid_moon.columns

Index(['HM_taxID', 'HM_name', 'HM_species_id', 'HM_species', 'HM_genus',
       'HM_family', 'HM_order', 'HM_class', 'HM_phylum', 'HM_kingdom'],
      dtype='object')

In [61]:
df_taxid_humanRef_lineage.columns

Index(['tax_id', 'species_id', 'name', 'species', 'genus', 'family', 'order',
       'class', 'phylum', 'kingdom'],
      dtype='object')

In [62]:
df_taxid_moon.columns = ['tax_id',  'name','species_id', 'species', 'genus', 'family', 'order',
       'class', 'phylum', 'kingdom']

In [65]:
df_taxid_moon.head()

,tax_id,name,species_id,species,genus,family,order,class,phylum,kingdom
index,,,,,,,,,,
141,471870,Bacteroides intestinalis DSM 17393,329854,Bacteroides intestinalis,Bacteroides,Bacteroidaceae,Bacteroidales,Bacteroidia,Bacteroidetes,Bacteria
142,927665,Parabacteroides goldsteinii DSM 19448 = WAL 12034,328812,Parabacteroides goldsteinii,Parabacteroides,Tannerellaceae,Bacteroidales,Bacteroidia,Bacteroidetes,Bacteria
145,469590,Bacteroides sp. 2_2_4,469590,Bacteroides sp. 2_2_4,Bacteroides,Bacteroidaceae,Bacteroidales,Bacteroidia,Bacteroidetes,Bacteria
149,1077285,Bacteroides faecis MAJ27,674529,Bacteroides faecis,Bacteroides,Bacteroidaceae,Bacteroidales,Bacteroidia,Bacteroidetes,Bacteria
150,817,Bacteroides fragilis,817,Bacteroides fragilis,Bacteroides,Bacteroidaceae,Bacteroidales,Bacteroidia,Bacteroidetes,Bacteria


In [66]:
df_taxid_moon1  = df_taxid_moon.drop_duplicates(subset=['tax_id'])
print(df_taxid_moon.shape, df_taxid_moon1.shape,)

(4765, 10) (428, 10)


In [68]:
#combine df_taxid_humanRef_lineage and df_taxid_moon1
df_taxid = pd.concat([df_taxid_humanRef_lineage,df_taxid_moon1],ignore_index=True, sort=True)
print(df_taxid.shape)

(32198, 10)


In [69]:
df_taxid = df_taxid.drop_duplicates(subset=['tax_id'])
print(df_taxid.shape)

(31798, 10)


In [72]:
df_taxid = df_taxid.loc[:,['tax_id',  'name','species_id', 'species', 'genus', 'family', 'order','class', 'phylum', 'kingdom']]

In [73]:
df_taxid.to_csv('/mnt/d/linux/W/NCBI/MoonRef20180810/20180822MoonBioWeb.taxLineageName',sep='\t',index=None)

In [74]:
df_species = df_taxid.loc[:,['species_id', 'species', 'genus', 'family', 'order', 'class', 'phylum', 'kingdom']]
df_species = df_species.drop_duplicates(subset=['species_id'])
print(df_species.shape)

(2715, 8)


In [75]:
df_species.to_csv('/mnt/d/linux/W/NCBI/MoonRef20180810/20180822MoonBioWeb.speciesLineageName',sep='\t',index=None)

In [82]:
df_moonbiotech3 = df_moonbiotech3.copy()

In [83]:
df_moonbiotech3['HM_subject'] = df_moonbiotech3['HM_subject'].apply(lambda x:x.split('.')[0])

In [84]:
df_moonbiotech3 = df_moonbiotech3.replace('nan',np.nan)
df_moonbiotech3.to_excel('/mnt/d/linux/W/NCBI/MoonRef20180810/20180822MoonBioWeb.moonLineages.xlsx',index=None,
                       columns=['all_IDs', 'seq', 'seq_len', 'HM_subject', 'HM_matchLength', 'HM_identical', 
                                'HM_qcover', 'HM_identity', 'HM_taxID', 'HM_name', 'HM_species_id', 'HM_species',
                                'HM_genus', 'HM_family', 'HM_order', 'HM_class', 'HM_phylum', 'HM_kingdom',
                               'Isolation sources code','Isolation place', 'Isolation source', 'Isolation medium',
                               'Cultivated medium', 'Amplified cultre medium', 'Cultivated TM', 'cultivated condition'])

## Get 16s sequences from Ezbio which are in the df_taxid

In [90]:
df_ezTax = pd.read_csv('/mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.taxa',sep='\t',header=None,dtype=str)

In [91]:
df_ezTax.shape

(51227, 2)

In [92]:
df_ezTax.head()

,0,1
0,AY570677,115547
1,GQ134545,77133
2,AB048217,113567
3,CP015971,1850526
4,AY771936,34034


In [93]:
st_taxBac = set(df_taxid['tax_id'])

In [94]:
df_ezTaxHM = df_ezTax[df_ezTax[1].apply(lambda x:x in st_taxBac)]
df_ezTaxHM.shape

(2579, 2)

In [102]:
df_ezTaxHM.to_csv('/mnt/d/linux/W/NCBI/Ezbio20180810/20180822HumanBacteria.taxa',sep='\t',header=None,index=None)

In [98]:
st_HM_accs = set(df_ezTaxHM[0])

In [101]:
fout = open('/mnt/d/linux/W/NCBI/Ezbio20180810/20180822HumanBacteria.fasta','w')
for _e in SeqIO.parse('/mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.fasta','fasta'):
    if _e.id.split('.')[0] in st_HM_accs:
        fout.write('>'+_e.id.split('.')[0]+'\n'+str(_e.seq)+'\n')
fout.close()